# Introdução

Neste documento iniciamos a parte de recuperacao de documentos PDF partindo do suposto que temos uma base de arquivos PDF para utilizar.

Neste preprocessamento, extraimos o texto dos pdfs para serem posteriormente processadas.

Nossas etapas consistem em:

* [Explicar etapas]

# Parte 0: Instalações

In [57]:
# %pip install PyPDF2

# Parte 1: Listagem dos PDFs

In [117]:
import os

In [118]:
pdf_folder_path = "./parsed_pdf"

In [119]:
filename_list = []
for filename in os.listdir(pdf_folder_path):
    if filename.endswith(".txt"):
        filename_list.append(os.path.join(pdf_folder_path, filename))

filename_list = sorted(filename_list)

In [120]:
filename_list

['./parsed_pdf/documento_11422_10324.txt',
 './parsed_pdf/documento_11422_13409.txt',
 './parsed_pdf/documento_11422_14547.txt',
 './parsed_pdf/documento_11422_20921.txt',
 './parsed_pdf/documento_11422_21245.txt',
 './parsed_pdf/documento_11422_3776.txt',
 './parsed_pdf/documento_11422_4587.txt',
 './parsed_pdf/documento_11422_8763.txt']

# Parte 2: Obtenção dos Metadados do Pantheon

In [121]:
import os
import json

In [122]:
metadata_folder_path = "./metadados_pantheon"

In [123]:
filename_metadata_list = []
for filename in os.listdir(metadata_folder_path):
    if filename.endswith(".json") and not filename.endswith("_x.json"):
        filename_metadata_list.append(os.path.join(metadata_folder_path, filename))

filename_metadata_list = sorted(filename_metadata_list)

In [124]:
filename_metadata_list 

['./metadados_pantheon/documento_11422_10324.json',
 './metadados_pantheon/documento_11422_13409.json',
 './metadados_pantheon/documento_11422_14547.json',
 './metadados_pantheon/documento_11422_20921.json',
 './metadados_pantheon/documento_11422_21245.json',
 './metadados_pantheon/documento_11422_3776.json',
 './metadados_pantheon/documento_11422_4587.json',
 './metadados_pantheon/documento_11422_8763.json']

In [125]:
metadata_dict = {}

for filename in filename_metadata_list:
    with open(filename) as f:
        metadata = json.load(f)

    _id = metadata["id"]
    
    metadata_dict[_id] = metadata

In [126]:
# metadata_dict

# Parte 3: Recuperação do arquivo txt

In [127]:
def retrieve_document(filename):
    with open(filename, "r") as file:
        file_text = file.read()
    return file_text

## Teste

In [128]:
full_text = retrieve_document(filename_list[0])
print(full_text[:300])


 
Análise da Relação dos Indicadores Lucro/Preço e Valor Contábil/Preço com o 
Retorno de Ações e Crescimento do Lucro das Empresas Brasileiras  
 
 
 
RESUMO  
 
 
As demonstrações contábeis são essenciais para precificar os ativos de uma 
empresa. Fama e French  (1992) demonstraram com modelos mu


# Parte 4: Tratamento do Texto em Blocos

* Essa parte é importante porque os blocos precisam de um tamanho máximo de 4096 tokens para serem processados pelos modelos.
* Não podemos simplesmente cortar os blocos na metade das frases.
* Em muitos dos pdfs já estamos garantidos pelo \n duplo, que deixa um espaço entre os parágrafos.
* Mas em alguns o texto é muito fluido e não possui linhas vazias.
* Também existe uma peculiaridade do PyPDF que ele não identifica texto corrido, então as linhas ficam quebradas exatamente na mudança de linha, o que nos impede de interpretar quebra de linha como mudança de parágrafo.

**Regras de Tratamento:**
1. Quebra de blocos por saltos de linha duplicados, indicando a quebra de um parágrafo.

In [129]:
import re

In [161]:
class TextParser:
    
    TOKEN_CTX_SIZE = 1000
    
    def parse_paragraph(self, text: str) -> list:
        text_split = text.split()
        if len(text_split) < self.TOKEN_CTX_SIZE:
            return [text]
        else:
            return text.split('.')

    def join_paragraphs(self, text_pieces: list) -> list:
        """Joins text pieces limiting by the TOKEN_CTX_SIZE"""

        final_text_pieces = []
        
        current_token_size = 0
        current_text = ""
        for piece in text_pieces:
            piece_tokens = piece.split()
                        
            if current_token_size + len(piece_tokens) > self.TOKEN_CTX_SIZE:
                final_text_pieces.append(current_text)
                current_token_size = 0
                current_text = ""
                
            current_token_size += len(piece_tokens)
            current_text += piece + "\n"                

        final_text_pieces.append(current_text)
        
        return final_text_pieces
    
    def parse_full_text(self, full_text: str):
        parsed_text = []
        
        for text in re.split(r'\n[ \t]+\n', full_text):
            blocks = self.parse_paragraph(text)
            parsed_text.extend(blocks)

        joined_pieces = self.join_paragraphs(parsed_text)

        return joined_pieces

## Teste:

In [162]:
text_parser = TextParser()

In [163]:
parsed_text = text_parser.parse_full_text(full_text)

In [164]:
[len(t) for t in parsed_text]

[5214, 5488, 5985, 6243, 5890, 5580, 5327, 5950, 6549, 322]

In [165]:
print(parsed_text[0])


Análise da Relação dos Indicadores Lucro/Preço e Valor Contábil/Preço com o 
Retorno de Ações e Crescimento do Lucro das Empresas Brasileiras  
 
RESUMO  
 
As demonstrações contábeis são essenciais para precificar os ativos de uma 
empresa. Fama e French  (1992) demonstraram com modelos multifatoriais 
que o tamanho e o valor contábil do patrimônio líquido e preço de mercado 
(B/P) são importantes para explicar os retornos das ações. Penman et al. 
(2018) mostraram que as variáveis desalavancadas e a alavanca gem 
financeira explicaram os retornos. O objetivo dessa dissertação é estudar a 
relação entre as variáveis contábeis e de mercado com o crescimento dos 
lucros e retorno das ações, bem como a relação entre a alavancagem 
financeira e as variáveis desalavanca das com o retorno das ações. Foi 
realizado um estudo cross -section  com dados de 161 ações de empresas 
dos índices IBRX100 e small caps da bolsa Brasileira, coletados durante o 
período de 2011 a 201 6. A regressão line

# Parte 5: Refinamento da Formatação dos Blocos

## Llama

In [166]:
# from langchain.chains.llm import LLMChain
# from langchain.prompts import PromptTemplate

In [167]:
# from langchain.llms import LlamaCpp
# from langchain.callbacks.manager import CallbackManager
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [168]:
# n_gpu_layers = 50  # Change this value based on your model and your GPU VRAM pool.
# n_batch = 2000  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [169]:
# llm = LlamaCpp(
#     model_path="/home/vinic/Projeto Final UFRJ/Arquivos Gerais/llama-2-13b-chat.Q5_K_M.gguf",
#     n_gpu_layers=n_gpu_layers,
#     n_batch=n_batch,
#     temperature=0.1,
#     n_ctx=2000,
#     max_tokens=2000,
#     top_p=1,
#     echo=False,
#     f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
#     callback_manager=callback_manager,
#     # verbose=True,
# )

## Template

In [170]:
# template = """
# '''
# {doc}
# '''

# O texto anterior foi extraído de um documento e pode conter formatações indevidas, como quebras entre palavras e quebras de linhas.
# Reescreva o texto como o original, apenas corrigindo os erros de formatação:
# """

In [171]:
# template = """
# O seguinte texto possui alguns erros de formatação, como quebras entre palavras e quebras de linhas indevidas:

# '''{doc}'''

# Escreva o texto novamente por completo, com o conteúdo igual ao original, mas corrigindo a formatação.
# Mantenha a língua do texto original.
# TEXTO CORRIGIDO:
# """

In [172]:
# template = """
# ### Instrução:
# O texto de entrada foi extraído de um documento PDF e está mal formatado.
# Reescreva o texto mantendo o conteúdo original mas com uma formatação melhor.
# Não omita nenhuma informação do texto e use a mesma língua do original.

# ### Entrada:
# {doc}

# ### Resposta:
# """

In [173]:
# template = """
# {doc}

# Reformate o texto anterior mantendo seu conteúdo e em português:
# """

In [174]:
# template = """
# '''
# {doc}
# '''

# O texto acima é um trecho de um documento e está mal formatado.
# Apenas reescreva o texto com o mesmo conteúdo, corrigindo a formatação e mantenha a língua como português.
# Resposta:
# """

In [175]:
# template = """
# O texto a seguir, dentro dos separadores ###, foi extraído de um documento pdf e contém alguns problemas de formatação e informações de rodapé inseridas indevidamente no meio do texto. Sua tarefa é reescrever esse mesmo texto, mantendo o conteúdo original, mas tratando esses problemas de formatação, como quebras de linhas indevidas durante o texto do parágrafo, quebra de palavras, e partes do rodapé inseridas indevidamente no meio do texto.

# ###
# {doc}
# ###

# Responda apenas com o texto corrido e corrigido, mantendo a mesma língua e palavras do texto original!
# Não faça resumos e nem pressuponha textos, precisa ser verídico ao texto original!
# Texto Corrigido:
# """

In [176]:
# template = """
# {doc}

# O texto acima foi extraído de um documento pdf e contém alguns problemas de formatação e informações de rodapé inseridas indevidamente no meio do texto. Sua tarefa é reescrever esse mesmo texto, mantendo o conteúdo original, mas tratando esses problemas de formatação, como quebras de linhas indevidas durante o texto do parágrafo, quebra de palavras, e partes do rodapé inseridas indevidamente no meio do texto.
# Responda apenas com o texto corrido e corrigido, mantendo a mesma língua e palavras do texto original!
# Não faça resumos e nem pressuponha textos, precisa ser verídico ao texto original!
# Texto Corrigido:
# """

In [177]:
# prompt = PromptTemplate.from_template(template)
# chain = LLMChain(llm=llm, prompt=prompt)

## Formatação

In [178]:
# print(' '.join(re.split(r'[ \t\n]+', parsed_text[0])))

In [179]:
# print(parsed_text[0][500:])

In [180]:
# chain.run(parsed_text[0][800:])

## GPT-3.5

In [181]:
openai_api_key = "xxxxxxx"

In [182]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage

In [190]:
chat = ChatOpenAI(
    model_name='gpt-3.5-turbo-16k',
    temperature = 0.0,
    openai_api_key = openai_api_key,         
    max_tokens=6000
)

In [200]:
class TextFormatter:
    template = """
    O texto passado pelo usuário foi extraído de um documento pdf e contém alguns problemas de formatação e informações de rodapé inseridas indevidamente no meio do texto. 
    Sua tarefa é reescrever esse mesmo texto, mantendo o conteúdo original, mas tratando esses problemas de formatação, como quebras de linhas indevidas durante o texto do parágrafo, quebra de palavras, e partes do rodapé inseridas indevidamente no meio do texto.
    Responda apenas com o texto corrido e corrigido, mantendo a mesma língua e palavras do texto original!
    Não faça resumos e nem pressuponha textos, precisa ser verídico ao texto original!
    """
    system_message_prompt = SystemMessagePromptTemplate.from_template(template)
    
    human_template = "{text}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

    chat_prompt = ChatPromptTemplate.from_messages(
        [system_message_prompt, human_message_prompt]
    )

    def format_text(self, text_sections):
        paragraphs = []
        
        print("Formatting Text: ", end="")
        
        for text in text_sections:
            print(".", end="")
            
            fixed_text = chat(self.chat_prompt.format_prompt(text=text).to_messages()).content
            fixed_paragraphs = fixed_text.split('\n')
            paragraphs.extend(fixed_paragraphs)

        paragraphs = [p for p in paragraphs if p != '']

        print(" finished!")
        
        return paragraphs

# Parte 6: Processamento Completo

In [201]:
text_parser = TextParser()

In [202]:
text_formatter = TextFormatter()

In [203]:
final_json = {}

In [195]:
formatted_documents_path = "./formatted_documents"

In [208]:
for filename in filename_list:
    _filename = filename.split('/')[-1].split('.')[0]
    _id = "-".join(_filename.split('_')[-2:])
    _id_underscore = "_".join(_filename.split('_')[-2:])
    
    _metadata = metadata_dict[_id]
    
    print(_id)
    print(_metadata["titulo"], "\n")

    # document retrieval
    document = retrieve_document(filename)

    # parse document - break in blocks
    parsed_text = text_parser.parse_full_text(document)
    
    # format document
    formatted_text = text_formatter.format_text(parsed_text)

    final_json[_id] = {
        **_metadata,
        "sentences": formatted_text
    }

    with open(f'{formatted_documents_path}/documento_' + _id_underscore + ".json", 'w', encoding='utf-8') as f:
        json.dump(final_json[_id], f, ensure_ascii=False)

11422-4587
Astrometria, efemérides e ocultações estelares de satélites irregulares e corpos do sistema solar exterior 

Formatting Text: .............

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


..

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


..........

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


.

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


....... finished!
11422-8763
Avaliação clínico e laboratorial das mães dos meninos com distrofia muscular de Duchenne 

Formatting Text: ......... finished!
